In [0]:
%run "./config"

In [0]:
spark.conf.set(
    "fs.azure.account.key.storekm01.dfs.core.windows.net",
    f'{key}'
)

In [0]:
display(dbutils.fs.ls("abfss://rawdata@storekm01.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-01/,2025-01-01/,0,1736093953000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-02/,2025-01-02/,0,1736093973000
abfss://rawdata@storekm01.dfs.core.windows.net/2025-01-03/,2025-01-03/,0,1736094000000


In [0]:
dbutils.widgets.text('Date','')
input_value= dbutils.widgets.get('Date')

In [0]:
import pyspark
# Create a spark session using getOrCreate() function 
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder.getOrCreate() 

In [0]:
df = spark_session.read.format( 
    "csv").option( 
    "header", True).load(f'{raw_folder_path}/customer 1.csv') 

In [0]:
df.show()

+----------+---------+-----+-----------+----------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
|CustomerID|NameStyle|Title|  FirstName|MiddleName|  LastName|Suffix|         CompanyName|         SalesPerson|        EmailAddress|              Phone|        PasswordHash|PasswordSalt|             rowguid|        ModifiedDate|
+----------+---------+-----+-----------+----------+----------+------+--------------------+--------------------+--------------------+-------------------+--------------------+------------+--------------------+--------------------+
|         1|    FALSE|  Mr.|    Orlando|        N.|       Gee|  NULL|        A Bike Store|adventure-works\p...|orlando0@adventur...|       245-555-0173|L/Rlwxzp4w7RWmEgX...|    1KjXYs4=|3f5ae95e-b87d-4ae...|2005-08-01T00:00:...|
|         2|    FALSE|  Mr.|      Keith|      NULL|    Harris|  NULL|  Progressive S

In [0]:
from pyspark.sql.functions import col,concat_ws, current_timestamp, lit, split
df_modified= df.withColumn('CustomerName', concat_ws('',col ("Title"), lit(' '), col("FirstName"),lit(' '), col("MiddleName"),lit(' '), col 
                ("LastName")))\
               .drop('PasswordHash','NameStyle','PasswordSalt','rowguid','ModifiedDate','Title','FirstName','LastName','MiddleName','Suffix')\
               .withColumn("Date",lit(input_value))

In [0]:
df_modified.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- SalesPerson: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- CustomerName: string (nullable = false)
 |-- Date: string (nullable = false)



In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, DateType

df_modified = (df_modified
               .withColumn("CustomerID", col("CustomerID").cast(IntegerType()))
               .withColumn("CompanyName", col("CompanyName").cast(StringType()))
               .withColumn("SalesPerson", col("SalesPerson").cast(StringType()))
               .withColumn("EmailAddress", col("EmailAddress").cast(StringType()))
               .withColumn("Phone", col("Phone").cast(StringType()))
               .withColumn("CustomerName", col("CustomerName").cast(StringType()))
               .withColumn("Date", col("Date").cast(DateType())))  # Cast Date to DateType


In [0]:
df_modified.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- SalesPerson: string (nullable = true)
 |-- EmailAddress: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- CustomerName: string (nullable = false)
 |-- Date: date (nullable = true)



In [0]:
df_modified.show()

+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----+
|CustomerID|         CompanyName|         SalesPerson|        EmailAddress|              Phone|        CustomerName|Date|
+----------+--------------------+--------------------+--------------------+-------------------+--------------------+----+
|         1|        A Bike Store|adventure-works\p...|orlando0@adventur...|       245-555-0173|  Mr. Orlando N. Gee|NULL|
|         2|  Progressive Sports|adventure-works\d...|keith0@adventure-...|       170-555-0127|   Mr. Keith  Harris|NULL|
|         3|Advanced Bike Com...|adventure-works\j...|donna0@adventure-...|       279-555-0130|Ms. Donna F. Carr...|NULL|
|         4|Modular Cycle Sys...|adventure-works\j...|janet1@adventure-...|       710-555-0173|  Ms. Janet M. Gates|NULL|
|         5|Metropolitan Spor...|adventure-works\shu0|lucy0@adventure-w...|       828-555-0186|Mr. Lucy  Harrington|NULL|
|         6|Aerobic Exer

In [0]:
customer_data_processed= df_modified.write.mode( "overwrite").option("header", "true").csv(f'{processed_folder_path}/customerdata')